#### Download all images from FTP to local filesystem

In [ ]:
import os

from dotenv import load_dotenv
from ftp import SingletonFTP

load_dotenv()
host = os.getenv('FTP_HOST')
username = os.getenv('FTP_USERNAME')
password = os.getenv('FTP_PASSWORD')


ftp = SingletonFTP(host, username, password)
ftp.download_all('/files/files-mailseth', 'tmp')

### extract metadata from filename

In [1]:
import astral
from astral.sun import sun
from pytz import timezone
location = astral.LocationInfo(
    "Oakland", "USA", "US/Pacific", 37.8044, -122.2712
)

In [3]:
from datetime import datetime
import os

image_dir = 'tmp'


image_files_list = [{'local_path': os.path.join(image_dir, f)}
               for f in os.listdir(image_dir) if f.endswith('.jpg')]

for output in image_files_list:
    filename = output['local_path'].split('/')[-1]
    camera_name, date, hour, minutes = filename.split('_')[3], filename.split(
        '_')[4], filename.split('_')[7], filename.split('_')[8].split('.')[0]
    seconds = filename.split('_')[8].split('.')[1][0:2]
    year, month, day = date.split('-')
    output['camera_name'] = camera_name
    datetime_obj = datetime(int(year), int(month), int(
        day), int(hour), int(minutes), int(seconds))
    output['datetime'] = datetime_obj
    # Use astral library to get sun information for the specific date and location
    s = sun(location.observer, date=output['datetime'].date(
    ), tzinfo=timezone('US/Pacific'))

    output['S3_key'] = camera_name + '_' + datetime_obj.strftime('%Y%m%d_%H%M%S') + '.jpg'

    # Classify the lighting based on the sun information
    if s['dawn'].time() <= output['datetime'].time() < s['sunrise'].time():
        output['lighting'] = 'dawn'
    elif s['sunrise'].time() <= output['datetime'].time() < s['sunset'].time():
        output['lighting'] = 'day'
    elif s['sunset'].time() <= output['datetime'].time() < s['dusk'].time():
        output['lighting'] = 'dusk'
    else:
        output['lighting'] = 'night'

### Detect objects in frames

In [34]:
import pickle

object_counts = {}
for image_file in image_files_list:
    status = image_file.get('detection_result', {}).get('status')
    if status:
        object_counts[status] = object_counts.get(status, 0) + 1
print(object_counts)

detection_counts = 0
for image_file in image_files_list:
    status = image_file.get('detection_result', None)
    if status:
        detection_counts = detection_counts + 1
print(detection_counts)

# with open('image_files_list_partial_202312011622.pkl', 'wb') as f:
#     pickle.dump(image_files_list, f)

# with open('output_list.pkl', 'rb') as f:
#     output_list = pickle.load(f)

{'COMPLETED': 2732, 'IN_PROGRESS': 3, 'IN_QUEUE': 160, 'FAILED': 2}
2897


In [28]:
from object_detection import object_detection_runpod
from tqdm import tqdm

from file_helper import write_file_to_s3, get_presigned_url

for iteration in tqdm(image_files_list):
    if 'detection_result' not in iteration:
        write_file_to_s3(iteration['S3_key'], iteration['local_path'])
        image_url = get_presigned_url(iteration['S3_key'])

        detection_result = object_detection_runpod(image_url)
        print('detection_result',detection_result)
        # detection_result_json = detection_result.pandas().xyxy[0].to_json()
        print('image_file', iteration['local_path'])
        print('detection_result', detection_result)
        iteration['detection_result'] = detection_result
        # output_list.append({
        #     'image_file': image_file,
        #     'detection_result': detection_result['output']
        # })
        # Explicitly delete the detection_result and call the garbage collector
        del detection_result

  7%|██▎                            | 2897/39072 [00:05<01:06, 540.24it/s]

<Response [200]>
{'delayTime': 1490, 'executionTime': 2922, 'id': 'sync-e2c99929-e077-4aaa-9f9a-1da37ccced36-u1', 'output': [[{'box': {'x1': 1338.14697265625, 'x2': 1470.8521728515625, 'y1': 1607.3499755859375, 'y2': 1991.7469482421875}, 'class': 0, 'confidence': 0.27170974016189575, 'name': 'person'}]], 'status': 'COMPLETED'}
detection_result {'delayTime': 1490, 'executionTime': 2922, 'id': 'sync-e2c99929-e077-4aaa-9f9a-1da37ccced36-u1', 'output': [[{'box': {'x1': 1338.14697265625, 'x2': 1470.8521728515625, 'y1': 1607.3499755859375, 'y2': 1991.7469482421875}, 'class': 0, 'confidence': 0.27170974016189575, 'name': 'person'}]], 'status': 'COMPLETED'}
image_file tmp/_files_files-mailseth_NorthStreetCam_2023-11-27_001_jpg_12_22.46[M][0@0][0].jpg
detection_result {'delayTime': 1490, 'executionTime': 2922, 'id': 'sync-e2c99929-e077-4aaa-9f9a-1da37ccced36-u1', 'output': [[{'box': {'x1': 1338.14697265625, 'x2': 1470.8521728515625, 'y1': 1607.3499755859375, 'y2': 1991.7469482421875}, 'class': 

  7%|██▎                            | 2897/39072 [00:08<01:42, 352.64it/s]


KeyboardInterrupt: 